# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

- **Consider a significance level of 5% for all tests.**

In [5]:
# import numpy and pandas
import pandas as pd
import numpy as np


# Challenge 1 - Independent Sample T-tests

In this challenge, we will be using the Pokemon dataset. Before applying statistical methods to this data, let's first examine the data.

To load the data, run the code below.

In [6]:
# Run this code:

pokemon = pd.read_csv('../pokemon.csv')

Let's start off by looking at the `head` function in the cell below.

In [7]:
# Your code here:

pokemon.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


The first thing we would like to do is compare the legendary Pokemon to the regular Pokemon. To do this, we should examine the data further. What is the count of legendary vs. non legendary Pokemons?

In [8]:
# Your code here:

pokemon["Legendary"].value_counts()

#no_legendary = pokemon[pokemon.Legendary==False]
#legendary = pokemon[pokemon.Legendary==True]

False    735
True      65
Name: Legendary, dtype: int64

Compute the mean and standard deviation of the total points for both legendary and non-legendary Pokemon.

In [9]:
# Your code here:

leg_tot = pokemon[pokemon['Legendary'] == True]['Total']
no_legendary_tot = pokemon[pokemon['Legendary'] == False]['Total']

print(leg_tot)
print(no_legendary_tot)


# Pokemon legendary total mean and standard deviation
#leg_mean, leg_std = np.mean(legendary.Total), np.std(legendary.Total)

# Pokemon non legendary total mean and standard deviation

#no_leg_mean, no_leg_std = np.mean(non_legendary.Total), np.std(non_legendary.Total)

156    580
157    580
158    580
162    680
163    780
      ... 
795    600
796    700
797    600
798    680
799    600
Name: Total, Length: 65, dtype: int64
0      318
1      405
2      525
3      625
4      309
      ... 
787    494
788    304
789    514
790    245
791    535
Name: Total, Length: 735, dtype: int64


In [10]:
# mean and std of pokemom legendary
leg_tot.describe()

count     65.000000
mean     637.384615
std       60.937389
min      580.000000
25%      580.000000
50%      600.000000
75%      680.000000
max      780.000000
Name: Total, dtype: float64

In [11]:
# mean and std of pokemom no-legendary
no_legendary_tot.describe()

count    735.000000
mean     417.213605
std      106.760417
min      180.000000
25%      324.000000
50%      425.000000
75%      498.000000
max      700.000000
Name: Total, dtype: float64

The computation of the mean might give us a clue regarding how the statistical test may turn out; However, it certainly does not prove whether there is a significant difference between the two groups.

In the cell below, use the `ttest_ind` function in `scipy.stats` to compare the the total points for legendary and non-legendary Pokemon. Since we do not have any information about the population, assume the variances are not equal.

In [12]:
# Your code here:

from scipy.stats import ttest_ind

ttest_ind(leg_tot, no_legendary_tot, equal_var = False)


Ttest_indResult(statistic=25.8335743895517, pvalue=9.357954335957446e-47)

What do you conclude from this test? Write your conclusions below.

In [13]:
# Your conclusions here:

P value is negative and way smaller than the alpha, so we can reject the null hypothesis.

SyntaxError: invalid syntax (<ipython-input-13-59247f4f9887>, line 3)

How about we try to compare the different types of pokemon? In the cell below, list the types of Pokemon from column `Type 1` and the count of each type.

In [ ]:
# Your code here:

#pokemon['Type 1'].unique()

pokemon['Type 1'].value_counts()

Since water is the largest group of Pokemon, compare the mean and standard deviation of water Pokemon to all other Pokemon.

In [19]:
# Your code here:

type_water = pokemon[pokemon["Type 1"] == "Water"] #.mean()

type_other = pokemon[pokemon["Type 1"] != "Water"]

type_water_mean= type_water['Total'].mean()
print("Mean Type Water", type_water_mean)

type_water_std= type_water['Total'].std()
print("Std Type Water", type_water_std)

type_other_mean= type_other['Total'].mean()
print("Mean Others", type_other_mean)

type_other_std= type_other['Total'].std()
print("Std Others", type_other_std)


Mean Type Water 430.45535714285717
Std Type Water 113.1882660643146
Mean Others 435.85901162790697
Std Others 121.0916823020807


Perform a hypothesis test comparing the mean of total points for water Pokemon to all non-water Pokemon. Assume the variances are equal. 

In [20]:
# Your code here:

from scipy import stats


#ttest_ind(type_water_mean, type_other, equal_var = False) #must be str, not float

ttest_ind(type_water['Total'], type_other['Total'])

#ttest_ind(leg_tot, no_legendary_tot, equal_var = False)

Ttest_indResult(statistic=-0.4418547448849676, pvalue=0.6587140317488793)

Write your conclusion below.

In [22]:
# Your conclusions here:

#as the pvalue is not smaller than my alpha 0.05, we can not reject the H0. 

# Challenge 2 - Matched Pairs Test

In this challenge we will compare dependent samples of data describing our Pokemon. Our goal is to see whether there is a significant difference between each Pokemon's defense and attack scores. Our hypothesis is that the defense and attack scores are equal. In the cell below, import the `ttest_rel` function from `scipy.stats` and compare the two columns to see if there is a statistically significant difference between them.

In [23]:
# Your code here:

from scipy.stats import ttest_rel

ttest_rel(pokemon['Attack'], pokemon['Defense'])

Ttest_relResult(statistic=4.325566393330478, pvalue=1.7140303479358558e-05)

Describe the results of the test in the cell below.

In [25]:
# Your conclusions here:

#I can reject the H0 because the pvalue is smaller than my alpha.


We are also curious about whether therer is a significant difference between the mean of special defense and the mean of special attack. Perform the hypothesis test in the cell below. 

In [26]:
pokemon.columns

Index(['#', 'Name', 'Type 1', 'Type 2', 'Total', 'HP', 'Attack', 'Defense',
       'Sp. Atk', 'Sp. Def', 'Speed', 'Generation', 'Legendary'],
      dtype='object')

In [27]:
# Your code here:

ttest_rel(pokemon['Sp. Atk'], pokemon['Sp. Def'])

Ttest_relResult(statistic=0.853986188453353, pvalue=0.3933685997548122)

Describe the results of the test in the cell below.

In [ ]:
# Your conclusions here:
We can´t reject the H0.


As you may recall, a two sample matched pairs test can also be expressed as a one sample test of the difference between the two dependent columns.

Import the `ttest_1samp` function and perform a one sample t-test of the difference between defense and attack. Test the hypothesis that the difference between the means is zero. Confirm that the results of the test are the same.

In [40]:
# Your code here:
    
from scipy.stats import ttest_1samp

ttest_1samp(pokemon['Defense'] - pokemon['Attack'], 0) #of the difference (-)


#https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.ttest_1samp.html
#didn´t get it even with documentation. The numbers are not equal...

Ttest_1sampResult(statistic=-4.325566393330478, pvalue=1.7140303479358558e-05)

# Bonus Challenge - The Chi-Square Test

The Chi-Square test is used to determine whether there is a statistically significant difference in frequencies. In other words, we are testing whether there is a relationship between categorical variables or rather when the variables are independent. This test is an alternative to Fisher's exact test and is used in scenarios where sample sizes are larger. However, with a large enough sample size, both tests produce similar results. Read more about the Chi Squared test [here](https://en.wikipedia.org/wiki/Chi-squared_test).

In the cell below, create a contingency table using `pd.crosstab` comparing whether a Pokemon is legenadary or not and whether the Type 1 of a Pokemon is water or not.

In [37]:
# Your code here:

pd.crosstab(index=[pokemon["Type 1"] == "Water"] , columns = pokemon["Legendary"]) #?

Legendary,False,True
Type 1,,
False,627,61
True,108,4


Perform a chi-squared test using the `chi2_contingency` function in `scipy.stats`. You can read the documentation of the function [here](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.chi2_contingency.html).

In [ ]:
# Your code here:





#class example for chi:
#observed_values=scipy.array([18,21,16,7,15])
#expected_values=scipy.array([22,19,44,8,16]) 

#stats.chisquare(observed_values, f_exp=expected_values)


Based on a 95% confidence, should we reject the null hypothesis?

In [ ]:
# Your answer here:

